# CS 4644: Final Project Models

Copyright (c) 2025 Ethan Nguyen-Tu

## Part 1: Setup

##### STEP 1: Mount Google Drive and Set Device for Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_path = "drive/MyDrive" # NOTE: Separated so that colab can access the '.kaggle' folder in your Google Drive for Kaggle API authentication
project_folder = drive_path + "/CS4644_FinalProject"

##### STEP 2: All Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import zipfile
from tqdm import tqdm
from torchvision import datasets, transforms, models
from PIL import Image

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from keras import layers
from keras import models
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

##### STEP 3: Seed for Reproducability

In [ ]:
SEED = 8

np.random.seed(SEED)
torch.manual_seed(SEED)
tf.random.set_seed(SEED)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(SEED)

##### STEP 4: General Helper Functions

In [ ]:
def train_torch(model, optimizer=None, criterion=nn.CrossEntropyLoss(), learning_rate=0.0001):
  model.to(DEVICE)

  if not optimizer:
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  train_losses = []
  train_accuracies = []

  val_losses = []
  val_accuracies = []

  for epoch in range(EPOCHS):

      model.train()

      train_correct = 0
      train_total = 0
      train_running_loss = 0.0

      for imgs, labels in train_loader:

          # STEP: Send tensors to device
          imgs = imgs.to(DEVICE)
          labels = labels.long().to(DEVICE)

          optimizer.zero_grad()

          outputs = model(imgs)
          loss = criterion(outputs, labels)

          loss.backward()
          optimizer.step()

          # STEP: Calculate training loss
          train_running_loss += loss.item()

          # STEP: Calculate training accuracy
          _, predictions = torch.max(outputs.data, 1)
          train_correct += (predictions == labels).sum().item()
          train_total += labels.size(0)

      # STEP: Calculate average training metrics
      avg_train_loss = train_running_loss / len(train_loader)
      train_losses.append(avg_train_loss)

      avg_train_acc = train_correct / train_total
      train_accuracies.append(avg_train_acc)


      model.eval()

      val_correct = 0
      val_total = 0
      val_running_loss = 0.0

      with torch.no_grad():
          for imgs, labels in val_loader:
              # STEP: Send tensors to device
              imgs = imgs.to(DEVICE)
              labels = labels.long().to(DEVICE)

              outputs = model(imgs)

              # STEP: Calculate validation loss
              val_loss = criterion(outputs, labels)
              val_running_loss += val_loss.item()

              # STEP: Calculate validation accuracy
              _, predictions = torch.max(outputs.data, 1)
              val_correct += (predictions == labels).sum().item()
              val_total += labels.size(0)

      # STEP: Calculate average validation metrics
      avg_val_loss = val_running_loss / len(val_loader)
      val_losses.append(avg_val_loss)

      avg_val_acc = val_correct / val_total
      val_accuracies.append(avg_val_acc)


      # STEP: Output Progress
      print(f"Epoch {epoch+1}/{EPOCHS} — accuracy: {avg_train_acc:.4f} - loss: {avg_train_loss:.4f} - val_accuracy {avg_val_acc:.4f} - val_loss: {avg_val_loss:.4f}")

  return pd.DataFrame({
      'accuracy': train_accuracies,
      'loss': train_losses,
      'val_accuracy': val_accuracies,
      'val_loss': val_losses
  })

In [ ]:
def test_pytorch(model, criterion=nn.CrossEntropyLoss()):
  model.eval()

  test_correct = 0
  test_total = 0
  test_running_loss = 0.0

  with torch.no_grad():
      for imgs, labels in test_loader:
          # STEP: Send tensors to device
          imgs = imgs.to(DEVICE)
          labels = labels.long().to(DEVICE)

          outputs = model(imgs)

          # STEP: Calculate validation loss
          test_loss = criterion(outputs, labels)
          test_running_loss += test_loss.item()

          # STEP: Calculate validation accuracy
          _, predictions = torch.max(outputs.data, 1)
          test_correct += (predictions == labels).sum().item()
          test_total += labels.size(0)

  # STEP: Calculate average validation metrics
  avg_test_loss = test_running_loss / len(test_loader)
  avg_test_acc = test_correct / test_total

  print(f"Test accuracy: {avg_test_acc:.4f} - Test loss: {avg_test_loss:.4f}")

## PART 2: Load the Datasets

In [ ]:
def zip_to_colab(zip_file_path, extract_dir_name):
  extract_dir = '/content/' + extract_dir_name + "/"
  os.makedirs(extract_dir, exist_ok=True)
  before_file_count = len(os.listdir(extract_dir))

  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extract_dir)

  files_extracted = len(os.listdir(extract_dir)) - before_file_count
  print(f"Files from {zip_file_path} extracted to: {extract_dir}")
  print("Number of files extracted:", files_extracted)
  return files_extracted

##### STEP 1: Load the "Human Faces" Dataset

Dataset Source: ["Human Faces" by Ashwin Gupta](https://www.kaggle.com/datasets/ashwingupta3012/human-faces/data)

In [ ]:
# zip_to_colab(project_folder + '/HumanFacesCleaned.zip', "HumanFacesImages")
# HUMANFACES_IMAGE_EXTENSIONS = {'.png', '.JPG', '.jpeg', '.jpg'}

##### STEP 2: Load the "Fake-Vs-Real-Faces (Hard)" Dataset

Dataset Source: ["Fake-Vs-Real-Faces (Hard)" by Hamza Boulahi](https://www.kaggle.com/datasets/hamzaboulahia/hardfakevsrealfaces)

In [ ]:
file_count = zip_to_colab(project_folder + '/RealImages.zip', 'FakeVsRealFacesImages')
file_count += zip_to_colab(project_folder + '/FakeImages.zip', 'FakeVsRealFacesImages')
assert len(os.listdir('/content/FakeVsRealFacesImages/')) == file_count # Check to make sure no files were overridden

Files from drive/MyDrive/CS4644_FinalProject/RealImages.zip extracted to: /content/FakeVsRealFacesImages/
Number of files extracted: 589
Files from drive/MyDrive/CS4644_FinalProject/FakeImages.zip extracted to: /content/FakeVsRealFacesImages/
Number of files extracted: 700


In [ ]:
FAKEVREAL_IMAGE_EXTENSIONS = {'.jpg'}
FAKEVREAL_IMAGE_PATH = '/content/FakeVsRealFacesImages/'

##### STEP 3: Load the "deepfake and real images" Dataset

Datset Source: ["deepfake and real images" by Manjil Karki](https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images)

In [ ]:
file_count = zip_to_colab(project_folder + '/Test_RealImages.zip', 'TestDataset')
file_count += zip_to_colab(project_folder + '/Test_FakeImages.zip', 'TestDataset')
assert len(os.listdir('/content/TestDataset/')) == file_count # Check to make sure no files were overridden

Files from drive/MyDrive/CS4644_FinalProject/Test_RealImages.zip extracted to: /content/TestDataset/
Number of files extracted: 129
Files from drive/MyDrive/CS4644_FinalProject/Test_FakeImages.zip extracted to: /content/TestDataset/
Number of files extracted: 129


In [ ]:
TEST_IMAGE_EXTENSIONS = {'.jpg'}
TEST_IMAGE_PATH = '/content/TestDataset/'

## PART 3: Get Test, Train, and Validation Sets

In [ ]:
def inspect_df(dataframe, rows=10):
  count_real = dataframe[dataframe['label'] == 1].shape[0]
  count_fake = dataframe[dataframe['label'] == 0].shape[0]
  assert count_real + count_fake == dataframe.shape[0]
  print("# of Real Images:", count_real)
  print("# of Fake Images:", count_fake)
  print("% Real:", (100 * count_real) // dataframe.shape[0])
  print("% Fake:", (100 * count_fake) // dataframe.shape[0])
  return dataframe.sample(frac=1, random_state=SEED).head(rows)

Step 1: Load the dataset labels into a dataframe.

In [ ]:
df_TestDataset = pd.read_csv(project_folder + "/test_data.csv")
df_TestDataset.head(10)

,images_id,label
0,fake_5234.jpg,0
1,fake_366.jpg,0
2,fake_4479.jpg,0
3,fake_2599.jpg,0
4,fake_577.jpg,0
5,fake_2475.jpg,0
6,fake_2431.jpg,0
7,fake_3544.jpg,0
8,fake_1061.jpg,0
9,fake_200.jpg,0


In [ ]:
df_RealVFake = pd.read_csv(project_folder + "/RealVFakeLabels.csv")
df_RealVFake.head(10)

,images_id,label
0,real_1,real
1,real_10,real
2,real_100,real
3,real_101,real
4,real_102,real
5,real_103,real
6,real_104,real
7,real_105,real
8,real_106,real
9,real_107,real


STEP 2: Replace the 'real' and 'fake' labelings with 1 and 0 respectively if needed.

In [ ]:
df_RealVFake.loc[df_RealVFake['label'] == 'real', 'label'] = 1
df_RealVFake.loc[df_RealVFake['label'] == 'fake', 'label'] = 0
inspect_df(df_RealVFake)

# of Real Images: 589
# of Fake Images: 700
% Real: 45
% Fake: 54


,images_id,label
1123,fake_580,0
1265,fake_78,0
385,real_446,1
955,fake_429,0
437,real_493,1
230,real_306,1
1107,fake_566,0
688,fake_189,0
111,real_2,1
223,real_30,1


STEP 3: Randomly separate the rows into 80% train, 10% validation and 10% test sets.

In [ ]:
# Shuffle Dataset
df_RealVFake_shuffled = df_RealVFake.sample(frac=1, random_state=SEED)

# Calculate the split index
separation_idx1 = int(0.8 * len(df_RealVFake_shuffled))
separation_idx2 = int(0.9 * len(df_RealVFake_shuffled))

# Split data into train and test sets
df_train_RealVFake = df_RealVFake_shuffled[:separation_idx1]
df_test_RealVFake = df_RealVFake_shuffled[separation_idx1:separation_idx2]
df_val_RealVFake = df_RealVFake_shuffled[separation_idx2:]

print("Train set shape:", df_train_RealVFake.shape)
print("Test set shape:", df_test_RealVFake.shape)
print("Validation set shape:", df_val_RealVFake.shape)

Train set shape: (1031, 2)
Test set shape: (129, 2)
Validation set shape: (129, 2)


In [ ]:
inspect_df(df_train_RealVFake)

# of Real Images: 470
# of Fake Images: 561
% Real: 45
% Fake: 54


,images_id,label
1097,fake_557,0
225,real_301,1
1260,fake_73,0
857,fake_340,0
468,real_520,1
1095,fake_555,0
232,real_308,1
292,real_362,1
1104,fake_563,0
640,fake_145,0


In [ ]:
inspect_df(df_test_RealVFake)

# of Real Images: 58
# of Fake Images: 71
% Real: 44
% Fake: 55


,images_id,label
481,real_532,1
744,fake_239,0
1009,fake_478,0
259,real_332,1
275,real_347,1
470,real_522,1
1050,fake_514,0
82,real_173,1
128,real_214,1
514,real_562,1


In [ ]:
inspect_df(df_val_RealVFake)

# of Real Images: 61
# of Fake Images: 68
% Real: 47
% Fake: 52


,images_id,label
809,fake_298,0
1109,fake_568,0
1038,fake_503,0
192,real_272,1
2,real_100,1
803,fake_292,0
742,fake_237,0
422,real_48,1
735,fake_230,0
1193,fake_643,0


In [ ]:
inspect_df(df_TestDataset)

# of Real Images: 129
# of Fake Images: 129
% Real: 50
% Fake: 50


,images_id,label
106,fake_4118.jpg,0
193,real_321.jpg,1
100,fake_4438.jpg,0
227,real_2071.jpg,1
30,fake_3354.jpg,0
25,fake_5083.jpg,0
253,real_4180.jpg,1
79,fake_3526.jpg,0
77,fake_4860.jpg,0
146,real_1616.jpg,1


## PART 4: Base Models

##### Reference Paper Keras Model

Original Paper Reference: https://philarchive.org/archive/SALCOR-3

In [ ]:
def get_keras_ref_model():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu',
    input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(512, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(2, activation='softmax')) # Changed to 2 from 4 in order to fit target shape

  return model

##### PyTorch Conversion

Convert the Reference Paper Keras Model to PyTorch

In [ ]:
# The Reference Model Converted

def get_pytorch_ref_model():
  conv2d_ks = 3 # Conv2d Kernel Size
  conv2d_pad = 1 # Conv2d Padding
  conv2d_s = 1 # Conv2d Stride

  maxpool2d_ks = 2 # MaxPool2d Kernel Size
  maxpool2d_s = 2 # MaxPool2d Stride

  return nn.Sequential(
      nn.Conv2d(3, 32, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(32, 64, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(64, 128, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(128, 256, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(256, 256, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(256, 512, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Flatten(),
      nn.Linear(8192, 512),
      nn.ReLU(),
      nn.Linear(512, 2), # Changed to 2 from 4 in order to fit target shape
      # nn.Softmax(), # Removed since loss applies sigmoid function internally
  )

## PART 5: Establishing the Baseline

Goal: Confirm that the Keras Model and PyTorch Conversion have the same accuracy on both the test and train set.

In [ ]:
IMG_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.0001 # Pg. 8 of "Classification of Real and Fake Human Faces Using Deep Learning" by Fatima Maher Salman and Samy S. Abu-Naser
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def basic_process_data(dataset, path, extension='.jpg'):
    keras_data = []
    torch_data = []
    labels = []

    for _, row in dataset.iterrows():
        image_path = ''.join([path, row["images_id"], extension])

        try:
            img = Image.open(image_path).resize((IMG_SIZE,IMG_SIZE))
            img = np.array(img) / float(IMG_SIZE)
            keras_data.append(img)
            img = torch.FloatTensor(img).permute(2, 0, 1)
            torch_data.append(img)
            labels.append(row["label"])

        except Exception as e:
            print(f"Error loading or processing image {image_path}: {e}")

    num_classes = len(set(labels))
    torch_labels = torch.tensor(labels, dtype=torch.float32).squeeze(0).to(DEVICE)
    torch_data = torch.stack(torch_data).to(DEVICE)

    return np.array(keras_data), to_categorical(np.array(labels), num_classes=num_classes), torch_data, torch_labels

In [ ]:
fakeVReal_TrainData, fakeVReal_TrainLabels, fakeVReal_TrainData_T, fakeVReal_TrainLabels_T = basic_process_data(df_train_RealVFake, FAKEVREAL_IMAGE_PATH)
fakeVReal_TestData, fakeVReal_TestLabels, fakeVReal_TestData_T, fakeVReal_TestLabels_T = basic_process_data(df_test_RealVFake, FAKEVREAL_IMAGE_PATH)
fakeVReal_ValData, fakeVReal_ValLabels, fakeVReal_ValData_T, fakeVReal_ValLabels_T = basic_process_data(df_val_RealVFake, FAKEVREAL_IMAGE_PATH)

In [ ]:
GENERATOR = torch.Generator()
GENERATOR.manual_seed(SEED)

##### STEP 1: Train the Base Keras Model

In [ ]:
model_reference = get_keras_ref_model()
model_reference.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

model_reference_results = model_reference.fit(fakeVReal_TrainData, fakeVReal_TrainLabels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_ValData, fakeVReal_ValLabels))
model_reference_history = pd.DataFrame(model_reference_results.history)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 224ms/step - accuracy: 0.5745 - loss: 0.6785 - val_accuracy: 0.6047 - val_loss: 0.6197
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 181ms/step - accuracy: 0.7754 - loss: 0.5695 - val_accuracy: 0.9767 - val_loss: 0.2934
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 188ms/step - accuracy: 0.9331 - loss: 0.2367 - val_accuracy: 0.9845 - val_loss: 0.0644
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.9815 - loss: 0.0611 - val_accuracy: 0.9922 - val_loss: 0.0179
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 178ms/step - accuracy: 0.9945 - loss: 0.0179 - val_accuracy: 0.9922 - val_loss: 0.0188
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 190ms/step - accuracy: 0.9989 - loss: 0.0094 - val_accuracy: 0.9922 - val_loss: 0.0189
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 0.9922 - val_loss: 0.0189
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 0

In [ ]:
model_reference_results.history

{'accuracy': [0.6149369478225708,
  0.8166828155517578,
  0.9447138905525208,
  0.9864209294319153,
  0.996120274066925,
  0.9990300536155701,
  1.0,
  1.0,
  1.0,
  1.0],
 'loss': [0.6655522584915161,
  0.5154090523719788,
  0.18452371656894684,
  0.0486675500869751,
  0.015066039748489857,
  0.007114512380212545,
  0.004098183009773493,
  0.0029981620609760284,
  0.0013950176071375608,
  0.0008736596792005002],
 'val_accuracy': [0.604651153087616,
  0.9767441749572754,
  0.9844961166381836,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918],
 'val_loss': [0.6196721196174622,
  0.29337531328201294,
  0.06437406688928604,
  0.017873490229249,
  0.018839584663510323,
  0.01890525035560131,
  0.018898339942097664,
  0.013096291571855545,
  0.01181737706065178,
  0.02047429420053959]}

In [ ]:
model_reference_history.to_csv(''.join(['/content/', project_folder, "/History/model_reference_history.csv"]))
model_reference_history

,accuracy,loss,val_accuracy,val_loss
0,0.614937,0.665552,0.604651,0.619672
1,0.816683,0.515409,0.976744,0.293375
2,0.944714,0.184524,0.984496,0.064374
3,0.986421,0.048668,0.992248,0.017873
4,0.996120,0.015066,0.992248,0.018840
5,0.999030,0.007115,0.992248,0.018905
6,1.000000,0.004098,0.992248,0.018898
7,1.000000,0.002998,0.992248,0.013096
8,1.000000,0.001395,0.992248,0.011817
9,1.000000,0.000874,0.992248,0.020474


In [ ]:
model_reference.save(''.join(['/content/', project_folder, "/Models/model_reference.keras"]))

##### STEP 2: Train the Base PyTorch Conversion Model

In [ ]:
train_dataset = torch.utils.data.TensorDataset(fakeVReal_TrainData_T, fakeVReal_TrainLabels_T)
test_dataset = torch.utils.data.TensorDataset(fakeVReal_TestData_T, fakeVReal_TestLabels_T)
val_dataset = torch.utils.data.TensorDataset(fakeVReal_ValData_T, fakeVReal_ValLabels_T)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=GENERATOR)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model_pytorch_conversion = get_pytorch_ref_model()
pytorch_conversion_history = train_torch(model_pytorch_conversion, learning_rate=LEARNING_RATE)

Epoch 1/10 — accuracy: 0.5441 - loss: 0.6888 - val_accuracy 0.5271 - val_loss: 0.7095
Epoch 2/10 — accuracy: 0.5441 - loss: 0.6856 - val_accuracy 0.5271 - val_loss: 0.7122
Epoch 3/10 — accuracy: 0.6479 - loss: 0.5952 - val_accuracy 0.8992 - val_loss: 0.3568
Epoch 4/10 — accuracy: 0.9079 - loss: 0.2520 - val_accuracy 0.9845 - val_loss: 0.0527
Epoch 5/10 — accuracy: 0.9602 - loss: 0.0987 - val_accuracy 1.0000 - val_loss: 0.0152
Epoch 6/10 — accuracy: 0.9816 - loss: 0.0542 - val_accuracy 1.0000 - val_loss: 0.0073
Epoch 7/10 — accuracy: 0.9835 - loss: 0.0445 - val_accuracy 0.9922 - val_loss: 0.0069
Epoch 8/10 — accuracy: 0.9913 - loss: 0.0213 - val_accuracy 1.0000 - val_loss: 0.0022
Epoch 9/10 — accuracy: 0.9981 - loss: 0.0064 - val_accuracy 1.0000 - val_loss: 0.0037
Epoch 10/10 — accuracy: 1.0000 - loss: 0.0036 - val_accuracy 1.0000 - val_loss: 0.0034


In [ ]:
pytorch_conversion_history.to_csv(''.join(['/content/', project_folder, "/History/pytorch_conversion_history.csv"]))
pytorch_conversion_history

,accuracy,loss,val_accuracy,val_loss
0,0.544132,0.688830,0.527132,0.709458
1,0.544132,0.685625,0.527132,0.712214
2,0.647915,0.595238,0.899225,0.356842
3,0.907856,0.251970,0.984496,0.052658
4,0.960233,0.098692,1.000000,0.015224
5,0.981571,0.054188,1.000000,0.007301
6,0.983511,0.044549,0.992248,0.006895
7,0.991271,0.021336,1.000000,0.002172
8,0.998060,0.006355,1.000000,0.003703
9,1.000000,0.003613,1.000000,0.003434


In [ ]:
torch.save(model_pytorch_conversion.state_dict(), ''.join(['/content/', project_folder, "/Models/pytorch_conversion.pth"]))

## PART 6: Optimized Models

#### Optimization - Learning Rate

The Keras Reference Model / PyTorch Conversion trained with 10x the learning rate of the baseline and 100x the learning rate of the baseline.

##### 1. Setup

In [ ]:
LEARNING_RATE_10x = 10 * LEARNING_RATE
LEARNING_RATE_100x = 100 * LEARNING_RATE

##### 2. Keras Model

###### 10x LR Keras

In [ ]:
keras_10x_LR = get_keras_ref_model()
keras_10x_LR.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_10x),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

keras_10x_LR_results = keras_10x_LR.fit(fakeVReal_TrainData, fakeVReal_TrainLabels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_ValData, fakeVReal_ValLabels))
keras_10x_LR_history = pd.DataFrame(keras_10x_LR_results.history)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 245ms/step - accuracy: 0.5088 - loss: 0.7318 - val_accuracy: 0.5271 - val_loss: 0.6808
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 185ms/step - accuracy: 0.6047 - loss: 0.6707 - val_accuracy: 0.5271 - val_loss: 0.6830
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - accuracy: 0.6140 - loss: 0.6657 - val_accuracy: 0.6589 - val_loss: 0.5821
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 178ms/step - accuracy: 0.7397 - loss: 0.5537 - val_accuracy: 0.8527 - val_loss: 0.4612
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 198ms/step - accuracy: 0.9279 - loss: 0.1996 - val_accuracy: 0.9845 - val_loss: 0.0860
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 205ms/step - accuracy: 0.9805 - loss: 0.0540 - val_accuracy: 0.9845 - val_loss: 0.0707
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 0.9927 - loss: 0.0196 - val_accuracy: 0.9845 - val_loss: 0.0673
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - accuracy: 0.9935 - loss: 0.0151 - val_accuracy: 0.

In [ ]:
keras_10x_LR_results.history

{'accuracy': [0.5247332453727722,
  0.6129971146583557,
  0.6391852498054504,
  0.7652764320373535,
  0.934044599533081,
  0.9864209294319153,
  0.99321049451828,
  0.9941803812980652,
  0.9990300536155701,
  0.9980601072311401],
 'loss': [0.7145230770111084,
  0.667194664478302,
  0.6497517228126526,
  0.5309072732925415,
  0.1837269365787506,
  0.04574223980307579,
  0.026443617418408394,
  0.018205123022198677,
  0.005105189513415098,
  0.007879451848566532],
 'val_accuracy': [0.5271317958831787,
  0.5271317958831787,
  0.6589147448539734,
  0.8527131676673889,
  0.9844961166381836,
  0.9844961166381836,
  0.9844961166381836,
  0.9767441749572754,
  0.9767441749572754,
  0.9844961166381836],
 'val_loss': [0.6808024644851685,
  0.6829755306243896,
  0.5821178555488586,
  0.46124884486198425,
  0.08599121868610382,
  0.07069525867700577,
  0.06732898950576782,
  0.09112127125263214,
  0.16572628915309906,
  0.13202263414859772]}

In [ ]:
keras_10x_LR_history.to_csv(''.join(['/content/', project_folder, "/History/keras_10x_LR_history.csv"]))
keras_10x_LR_history

,accuracy,loss,val_accuracy,val_loss
0,0.524733,0.714523,0.527132,0.680802
1,0.612997,0.667195,0.527132,0.682976
2,0.639185,0.649752,0.658915,0.582118
3,0.765276,0.530907,0.852713,0.461249
4,0.934045,0.183727,0.984496,0.085991
5,0.986421,0.045742,0.984496,0.070695
6,0.993210,0.026444,0.984496,0.067329
7,0.994180,0.018205,0.976744,0.091121
8,0.999030,0.005105,0.976744,0.165726
9,0.998060,0.007879,0.984496,0.132023


In [ ]:
keras_10x_LR.save(''.join(['/content/', project_folder, "/Models/keras_10x_LR.keras"]))

###### 100x LR Keras

In [ ]:
keras_100x_LR = get_keras_ref_model()
keras_100x_LR.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_100x),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

keras_100x_LR_results = keras_100x_LR.fit(fakeVReal_TrainData, fakeVReal_TrainLabels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_ValData, fakeVReal_ValLabels))
keras_100x_LR_history = pd.DataFrame(keras_100x_LR_results.history)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 233ms/step - accuracy: 0.5142 - loss: 72.7600 - val_accuracy: 0.5271 - val_loss: 0.6944
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 176ms/step - accuracy: 0.4836 - loss: 0.7339 - val_accuracy: 0.5271 - val_loss: 0.6930
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 183ms/step - accuracy: 0.5252 - loss: 0.6945 - val_accuracy: 0.5271 - val_loss: 0.6924
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 183ms/step - accuracy: 0.5326 - loss: 0.6920 - val_accuracy: 0.5271 - val_loss: 0.6927
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 200ms/step - accuracy: 0.5326 - loss: 0.6918 - val_accuracy: 0.5271 - val_loss: 0.6930
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 184ms/step - accuracy: 0.5326 - loss: 0.6919 - val_accuracy: 0.5271 - val_loss: 0.6928
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - accuracy: 0.5326 - loss: 0.6918 - val_accuracy: 0.5271 - val_loss: 0.6970
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 220ms/step - accuracy: 0.5326 - loss: 0.6937 - val_accuracy: 0

In [ ]:
keras_100x_LR_results.history

{'accuracy': [0.5160039067268372,
  0.4723569452762604,
  0.5208535194396973,
  0.5441319346427917,
  0.5441319346427917,
  0.5441319346427917,
  0.5441319346427917,
  0.5441319346427917,
  0.5441319346427917,
  0.5441319346427917],
 'loss': [42.48752212524414,
  0.7226982712745667,
  0.6940058469772339,
  0.6908707022666931,
  0.6903287172317505,
  0.6903913021087646,
  0.689769983291626,
  0.6915279030799866,
  0.6899444460868835,
  0.6896544694900513],
 'val_accuracy': [0.5271317958831787,
  0.5271317958831787,
  0.5271317958831787,
  0.5271317958831787,
  0.5271317958831787,
  0.5271317958831787,
  0.5271317958831787,
  0.5271317958831787,
  0.5271317958831787,
  0.5271317958831787],
 'val_loss': [0.6943789720535278,
  0.6930347084999084,
  0.6923968195915222,
  0.6926637291908264,
  0.6929847598075867,
  0.6928154230117798,
  0.6969687938690186,
  0.6919746398925781,
  0.6922745108604431,
  0.6927060484886169]}

In [ ]:
keras_100x_LR_history.to_csv(''.join(['/content/', project_folder, "/History/keras_100x_LR_history.csv"]))
keras_100x_LR_history

,accuracy,loss,val_accuracy,val_loss
0,0.516004,42.487522,0.527132,0.694379
1,0.472357,0.722698,0.527132,0.693035
2,0.520854,0.694006,0.527132,0.692397
3,0.544132,0.690871,0.527132,0.692664
4,0.544132,0.690329,0.527132,0.692985
5,0.544132,0.690391,0.527132,0.692815
6,0.544132,0.689770,0.527132,0.696969
7,0.544132,0.691528,0.527132,0.691975
8,0.544132,0.689944,0.527132,0.692275
9,0.544132,0.689654,0.527132,0.692706


In [ ]:
keras_100x_LR.save(''.join(['/content/', project_folder, "/Models/keras_100x_LR.keras"]))

##### 3. PyTorch Model

###### 10x LR PyTorch

In [ ]:
pyTorch_10x_LR = get_pytorch_ref_model()
pytorch_10X_LR_history = train_torch(pyTorch_10x_LR, learning_rate=LEARNING_RATE_10x)

Epoch 1/10 — accuracy: 0.5286 - loss: 0.6939 - val_accuracy 0.5271 - val_loss: 0.7109
Epoch 2/10 — accuracy: 0.5529 - loss: 0.6855 - val_accuracy 0.5349 - val_loss: 0.7242
Epoch 3/10 — accuracy: 0.7168 - loss: 0.5094 - val_accuracy 0.8915 - val_loss: 0.1613
Epoch 4/10 — accuracy: 0.9282 - loss: 0.1879 - val_accuracy 0.9845 - val_loss: 0.0340
Epoch 5/10 — accuracy: 0.9758 - loss: 0.0674 - val_accuracy 0.9922 - val_loss: 0.0407
Epoch 6/10 — accuracy: 0.9855 - loss: 0.0484 - val_accuracy 0.9922 - val_loss: 0.0192
Epoch 7/10 — accuracy: 0.9942 - loss: 0.0328 - val_accuracy 0.9845 - val_loss: 0.0300
Epoch 8/10 — accuracy: 0.9952 - loss: 0.0158 - val_accuracy 0.9922 - val_loss: 0.0130
Epoch 9/10 — accuracy: 0.9961 - loss: 0.0097 - val_accuracy 0.9922 - val_loss: 0.0178
Epoch 10/10 — accuracy: 0.9971 - loss: 0.0107 - val_accuracy 0.9922 - val_loss: 0.0066


In [ ]:
pytorch_10X_LR_history.to_csv(''.join(['/content/', project_folder, "/History/pytorch_10X_LR_history.csv"]))
pytorch_10X_LR_history

,accuracy,loss,val_accuracy,val_loss
0,0.528613,0.693946,0.527132,0.710865
1,0.552861,0.685452,0.534884,0.724214
2,0.716780,0.509352,0.891473,0.161299
3,0.928225,0.187892,0.984496,0.034047
4,0.975752,0.067390,0.992248,0.040729
5,0.985451,0.048428,0.992248,0.019208
6,0.994180,0.032762,0.984496,0.029970
7,0.995150,0.015821,0.992248,0.013005
8,0.996120,0.009654,0.992248,0.017750
9,0.997090,0.010666,0.992248,0.006624


In [ ]:
torch.save(pyTorch_10x_LR.state_dict(), ''.join(['/content/', project_folder, "/Models/pytorch_10X_LR.pth"]))

###### 100x LR PyTorch

In [ ]:
pyTorch_100x_LR = get_pytorch_ref_model()
pytorch_100X_LR_history = train_torch(pyTorch_100x_LR, learning_rate=LEARNING_RATE_100x)

Epoch 1/10 — accuracy: 0.4879 - loss: 284.1943 - val_accuracy 0.5039 - val_loss: 0.6895
Epoch 2/10 — accuracy: 0.5577 - loss: 0.7077 - val_accuracy 0.5271 - val_loss: 0.6673
Epoch 3/10 — accuracy: 0.5684 - loss: 0.6756 - val_accuracy 0.5271 - val_loss: 0.6992
Epoch 4/10 — accuracy: 0.6237 - loss: 0.6644 - val_accuracy 0.5659 - val_loss: 0.6688
Epoch 5/10 — accuracy: 0.7236 - loss: 0.5230 - val_accuracy 0.7674 - val_loss: 0.3899
Epoch 6/10 — accuracy: 0.8758 - loss: 0.2841 - val_accuracy 0.9147 - val_loss: 0.1520
Epoch 7/10 — accuracy: 0.9195 - loss: 0.1841 - val_accuracy 0.9380 - val_loss: 0.0851
Epoch 8/10 — accuracy: 0.9641 - loss: 0.0823 - val_accuracy 0.9612 - val_loss: 0.0617
Epoch 9/10 — accuracy: 0.9796 - loss: 0.0539 - val_accuracy 0.9922 - val_loss: 0.0231
Epoch 10/10 — accuracy: 0.9806 - loss: 0.0529 - val_accuracy 0.9922 - val_loss: 0.0124


In [ ]:
pytorch_100X_LR_history.to_csv(''.join(['/content/', project_folder, "/History/pytorch_100X_LR_history.csv"]))
pytorch_100X_LR_history

,accuracy,loss,val_accuracy,val_loss
0,0.487876,284.194304,0.503876,0.689490
1,0.557711,0.707747,0.527132,0.667313
2,0.568380,0.675596,0.527132,0.699199
3,0.623666,0.664388,0.565891,0.668827
4,0.723569,0.523041,0.767442,0.389926
5,0.875849,0.284079,0.914729,0.152018
6,0.919496,0.184075,0.937984,0.085081
7,0.964113,0.082305,0.961240,0.061726
8,0.979631,0.053869,0.992248,0.023138
9,0.980601,0.052932,0.992248,0.012412


In [ ]:
torch.save(pyTorch_100x_LR.state_dict(), ''.join(['/content/', project_folder, "/Models/pytorch_100X_LR.pth"]))

#### Optimization - Extra Layer

The Keras Reference Model / PyTorch Conversion with an additional convolution layer followed by relu and maxpool2d.

##### 1. Keras Model

In [ ]:
keras_additional_conv = models.Sequential()
keras_additional_conv.add(layers.Conv2D(32, (3, 3), activation='relu',
  input_shape=(256, 256, 3)))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(64, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(128, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(256, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(256, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(512, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(1024, (3, 3), activation='relu', padding="same"))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Flatten())
keras_additional_conv.add(layers.Dense(1024, activation='relu'))
keras_additional_conv.add(layers.Dense(2, activation='softmax')) # Changed to 2 from 4 in order to fit target shape

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
keras_additional_conv.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

keras_additional_conv_results = keras_additional_conv.fit(fakeVReal_TrainData, fakeVReal_TrainLabels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_ValData, fakeVReal_ValLabels))
keras_additional_conv_history = pd.DataFrame(keras_additional_conv_results.history)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 231ms/step - accuracy: 0.5409 - loss: 0.6849 - val_accuracy: 0.5271 - val_loss: 0.6552
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 196ms/step - accuracy: 0.7545 - loss: 0.5907 - val_accuracy: 0.9302 - val_loss: 0.2487
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 200ms/step - accuracy: 0.9290 - loss: 0.1908 - val_accuracy: 0.9767 - val_loss: 0.0686
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 197ms/step - accuracy: 0.9767 - loss: 0.0620 - val_accuracy: 0.9922 - val_loss: 0.0101
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 197ms/step - accuracy: 0.9979 - loss: 0.0141 - val_accuracy: 1.0000 - val_loss: 0.0080
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 207ms/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 0.9922 - val_loss: 0.0061
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 202ms/step - accuracy: 0.9987 - loss: 0.0042 - val_accuracy: 1.0000 - val_loss: 0.0063
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 191ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 0.

In [ ]:
keras_additional_conv_results.history

{'accuracy': [0.5693501234054565,
  0.8225024342536926,
  0.941804051399231,
  0.9844810962677002,
  0.997090220451355,
  1.0,
  0.9980601072311401,
  1.0,
  0.9990300536155701,
  1.0],
 'loss': [0.6766416430473328,
  0.519621729850769,
  0.15263842046260834,
  0.0492376983165741,
  0.012193153612315655,
  0.003924422897398472,
  0.005272210109978914,
  0.0024167178198695183,
  0.002584159141406417,
  0.00155644491314888],
 'val_accuracy': [0.5271317958831787,
  0.930232584476471,
  0.9767441749572754,
  0.9922480583190918,
  1.0,
  0.9922480583190918,
  1.0,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918],
 'val_loss': [0.6551749110221863,
  0.24872024357318878,
  0.06860291212797165,
  0.010133503004908562,
  0.008006172254681587,
  0.006121780723333359,
  0.006338406819850206,
  0.01796690560877323,
  0.008753921836614609,
  0.012596452608704567]}

In [ ]:
keras_additional_conv_history.to_csv(''.join(['/content/', project_folder, "/History/keras_additional_conv_history.csv"]))
keras_additional_conv_history

,accuracy,loss,val_accuracy,val_loss
0,0.569350,0.676642,0.527132,0.655175
1,0.822502,0.519622,0.930233,0.248720
2,0.941804,0.152638,0.976744,0.068603
3,0.984481,0.049238,0.992248,0.010134
4,0.997090,0.012193,1.000000,0.008006
5,1.000000,0.003924,0.992248,0.006122
6,0.998060,0.005272,1.000000,0.006338
7,1.000000,0.002417,0.992248,0.017967
8,0.999030,0.002584,0.992248,0.008754
9,1.000000,0.001556,0.992248,0.012596


In [ ]:
keras_additional_conv.save(''.join(['/content/', project_folder, "/Models/keras_additional_conv.keras"]))

##### 2. PyTorch Model

In [ ]:
def get_pytorch_ref_model_more_layers():
  conv2d_ks = 3 # Conv2d Kernel Size
  conv2d_pad = 1 # Conv2d Padding
  conv2d_s = 1 # Conv2d Stride

  maxpool2d_ks = 2 # MaxPool2d Kernel Size
  maxpool2d_s = 2 # MaxPool2d Stride

  return nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(32, 64, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(64, 128, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(128, 256, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(256, 256, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(256, 512, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(512, 1024, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Flatten(),
    nn.Linear(4096, 512),
    nn.ReLU(),
    nn.Linear(512, 2), # Changed to 2 from 4 in order to fit target shape
    # nn.Softmax(), # Removed since loss applies sigmoid function internally
  )

In [ ]:
pyTorch_additional_conv = get_pytorch_ref_model_more_layers()
pyTorch_additional_conv_history = train_torch(pyTorch_additional_conv, learning_rate=LEARNING_RATE)

Epoch 1/10 — accuracy: 0.5383 - loss: 0.6911 - val_accuracy 0.5271 - val_loss: 0.7125
Epoch 2/10 — accuracy: 0.5441 - loss: 0.6900 - val_accuracy 0.5271 - val_loss: 0.7116
Epoch 3/10 — accuracy: 0.5500 - loss: 0.6658 - val_accuracy 0.8992 - val_loss: 0.6288
Epoch 4/10 — accuracy: 0.8671 - loss: 0.3481 - val_accuracy 0.8915 - val_loss: 0.2651
Epoch 5/10 — accuracy: 0.9321 - loss: 0.2254 - val_accuracy 0.9845 - val_loss: 0.0713
Epoch 6/10 — accuracy: 0.9806 - loss: 0.0653 - val_accuracy 1.0000 - val_loss: 0.0081
Epoch 7/10 — accuracy: 0.9903 - loss: 0.0321 - val_accuracy 0.9922 - val_loss: 0.0139
Epoch 8/10 — accuracy: 0.9971 - loss: 0.0180 - val_accuracy 0.9922 - val_loss: 0.0255
Epoch 9/10 — accuracy: 0.9952 - loss: 0.0209 - val_accuracy 0.9922 - val_loss: 0.0397
Epoch 10/10 — accuracy: 0.9961 - loss: 0.0091 - val_accuracy 0.9922 - val_loss: 0.0324


In [ ]:
pyTorch_additional_conv_history.to_csv(''.join(['/content/', project_folder, "/History/pytorch_additional_conv_history.csv"]))
pyTorch_additional_conv_history

,accuracy,loss,val_accuracy,val_loss
0,0.538312,0.691131,0.527132,0.712493
1,0.544132,0.690017,0.527132,0.711606
2,0.549952,0.665772,0.899225,0.628778
3,0.867119,0.348142,0.891473,0.265051
4,0.932105,0.225409,0.984496,0.071252
5,0.980601,0.065344,1.000000,0.008105
6,0.990301,0.032124,0.992248,0.013883
7,0.997090,0.017952,0.992248,0.025453
8,0.995150,0.020947,0.992248,0.039700
9,0.996120,0.009069,0.992248,0.032373


In [ ]:
torch.save(pyTorch_additional_conv.state_dict(), ''.join(['/content/', project_folder, "/Models/pyTorch_additional_conv.pth"]))

#### Optimization - More Image Transforms

Same as the reference model, but the images transformations are added to the model's training data.


##### 1. Setup

In [ ]:
transform_tensor = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [ ]:
transform_numpy = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=15,
    brightness_range=[0.9, 1.1],
    channel_shift_range=0.1,
    rescale=1./255,
)

In [ ]:
def process_data_with_transforms(dataset, path):
    torch_data = []
    keras_data = []
    labels = []

    for _, row in dataset.iterrows():
        image_path = ''.join([path, row["images_id"], '.jpg'])

        try:
            img = Image.open(image_path)
            torch_data.append(transform_tensor(img))
            keras_data.append(np.array(img.resize((IMG_SIZE, IMG_SIZE))))
            labels.append(row["label"])

        except Exception as e:
            print(f"Error loading or processing image {image_path}: {e}")

    torch_labels = torch.tensor(labels, dtype=torch.long).to(DEVICE)
    torch_data = torch.stack(torch_data).to(DEVICE)

    num_classes = len(set(labels))
    keras_generator = transform_numpy.flow(np.stack(keras_data), y=to_categorical(np.array(labels), num_classes=num_classes), batch_size=BATCH_SIZE)

    return keras_generator, torch_data, torch_labels

In [ ]:
keras_generator, fakeVReal_Transforms_TrainData_T, fakeVReal_Transforms_TrainLabels_T = process_data_with_transforms(df_train_RealVFake, FAKEVREAL_IMAGE_PATH)

##### 2. Keras Model

In [ ]:
keras_imageTransforms = get_keras_ref_model()
keras_imageTransforms.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

keras_imageTransforms_results = keras_imageTransforms.fit(keras_generator, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_ValData, fakeVReal_ValLabels))
keras_imageTransforms_history = pd.DataFrame(keras_imageTransforms_results.history)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.5511 - loss: 0.6867 - val_accuracy: 0.5271 - val_loss: 0.6777
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.6324 - loss: 0.6509 - val_accuracy: 0.4729 - val_loss: 0.7309
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.5986 - loss: 0.6592 - val_accuracy: 0.7907 - val_loss: 0.5649
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.7287 - loss: 0.4958 - val_accuracy: 0.8837 - val_loss: 0.4153
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.8589 - loss: 0.3333 - val_accuracy: 0.9612 - val_loss: 0.1542
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.9397 - loss: 0.1638 - val_accuracy: 1.0000 - val_loss: 0.0524
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.9698 - loss: 0.0846 - val_accuracy: 0.9922 - val_loss: 0.0266
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.9919 - loss: 0.0405 - val_accuracy: 0.9922 - val_loss:

In [ ]:
keras_imageTransforms_results.history

{'accuracy': [0.5441319346427917,
  0.6411251425743103,
  0.668283224105835,
  0.7419980764389038,
  0.8933074474334717,
  0.9582929015159607,
  0.9796314239501953,
  0.9883608222007751,
  0.9912706017494202,
  0.99321049451828],
 'loss': [0.6859830617904663,
  0.647864580154419,
  0.6185535788536072,
  0.47547346353530884,
  0.2771652638912201,
  0.13279841840267181,
  0.06468268483877182,
  0.04074510559439659,
  0.03202630206942558,
  0.02131306566298008],
 'val_accuracy': [0.5271317958831787,
  0.4728682041168213,
  0.7906976938247681,
  0.8837209343910217,
  0.961240291595459,
  1.0,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918],
 'val_loss': [0.6776583790779114,
  0.7309403419494629,
  0.564941942691803,
  0.4153165817260742,
  0.15421269834041595,
  0.05243030935525894,
  0.02657330222427845,
  0.019978919997811317,
  0.03029737062752247,
  0.022865157574415207]}

In [ ]:
keras_imageTransforms_history.to_csv(''.join(['/content/', project_folder, "/History/keras_imageTransforms_history.csv"]))
keras_imageTransforms_history

,accuracy,loss,val_accuracy,val_loss
0,0.544132,0.685983,0.527132,0.677658
1,0.641125,0.647865,0.472868,0.730940
2,0.668283,0.618554,0.790698,0.564942
3,0.741998,0.475473,0.883721,0.415317
4,0.893307,0.277165,0.961240,0.154213
5,0.958293,0.132798,1.000000,0.052430
6,0.979631,0.064683,0.992248,0.026573
7,0.988361,0.040745,0.992248,0.019979
8,0.991271,0.032026,0.992248,0.030297
9,0.993210,0.021313,0.992248,0.022865


In [ ]:
keras_imageTransforms.save(''.join(['/content/', project_folder, "/Models/keras_imageTransforms.keras"]))

##### 4. PyTorch Model

In [ ]:
train_Transforms_dataset = torch.utils.data.TensorDataset(fakeVReal_Transforms_TrainData_T, fakeVReal_Transforms_TrainLabels_T)

train_loader = DataLoader(train_Transforms_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=GENERATOR)

In [ ]:
pyTorch_imageTransforms = get_pytorch_ref_model()
pyTorch_imageTransforms_history = train_torch(pyTorch_imageTransforms, learning_rate=LEARNING_RATE)

Epoch 1/10 — accuracy: 0.5674 - loss: 0.6815 - val_accuracy 0.5271 - val_loss: 0.7027
Epoch 2/10 — accuracy: 0.7468 - loss: 0.5024 - val_accuracy 0.8372 - val_loss: 0.5972
Epoch 3/10 — accuracy: 0.9408 - loss: 0.1590 - val_accuracy 0.8760 - val_loss: 0.4859
Epoch 4/10 — accuracy: 0.9631 - loss: 0.0987 - val_accuracy 0.9457 - val_loss: 0.4279
Epoch 5/10 — accuracy: 0.9845 - loss: 0.0516 - val_accuracy 0.9612 - val_loss: 0.3764
Epoch 6/10 — accuracy: 0.9952 - loss: 0.0229 - val_accuracy 0.9535 - val_loss: 0.3043
Epoch 7/10 — accuracy: 0.9971 - loss: 0.0136 - val_accuracy 0.9845 - val_loss: 0.2274
Epoch 8/10 — accuracy: 0.9971 - loss: 0.0083 - val_accuracy 0.9767 - val_loss: 0.2170
Epoch 9/10 — accuracy: 0.9981 - loss: 0.0054 - val_accuracy 0.9690 - val_loss: 0.1759
Epoch 10/10 — accuracy: 1.0000 - loss: 0.0026 - val_accuracy 0.9535 - val_loss: 0.1730


In [ ]:
pyTorch_imageTransforms_history.to_csv(''.join(['/content/', project_folder, "/History/pyTorch_imageTransforms_history.csv"]))
pyTorch_imageTransforms_history

,accuracy,loss,val_accuracy,val_loss
0,0.567410,0.681479,0.527132,0.702678
1,0.746848,0.502354,0.837209,0.597151
2,0.940834,0.158980,0.875969,0.485895
3,0.963143,0.098739,0.945736,0.427938
4,0.984481,0.051587,0.961240,0.376369
5,0.995150,0.022891,0.953488,0.304323
6,0.997090,0.013631,0.984496,0.227418
7,0.997090,0.008284,0.976744,0.216999
8,0.998060,0.005351,0.968992,0.175919
9,1.000000,0.002562,0.953488,0.172967


In [ ]:
torch.save(pyTorch_imageTransforms.state_dict(), ''.join(['/content/', project_folder, "/Models/pyTorch_imageTransforms.pth"]))

## Part 7: Test Models (Original Dataset)

##### STEP 1: Setup

1. Reset Parameters to Baseline if needed.

In [ ]:
IMG_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.0001 # Pg. 8 of "Classification of Real and Fake Human Faces Using Deep Learning" by Fatima Maher Salman and Samy S. Abu-Naser
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### STEP 2: Evaluate the Keras Models

In [ ]:
keras_reference = load_model(''.join([project_folder, "/Models/model_reference.keras"]))
loss, accuracy = keras_reference.evaluate(fakeVReal_TestData, fakeVReal_TestLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.9918 - loss: 0.0073
Test Loss: 0.006959858816117048
Test Accuracy: 0.9922480583190918


In [ ]:
keras_10xLR = load_model(''.join([project_folder, "/Models/keras_10x_LR.keras"]))
loss, accuracy = keras_10xLR.evaluate(fakeVReal_TestData, fakeVReal_TestLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.9701 - loss: 0.0927
Test Loss: 0.0742759257555008
Test Accuracy: 0.9767441749572754


In [ ]:
keras_100xLR = load_model(''.join([project_folder, "/Models/keras_100x_LR.keras"]))
loss, accuracy = keras_100xLR.evaluate(fakeVReal_TestData, fakeVReal_TestLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.5398 - loss: 0.6902
Test Loss: 0.688061535358429
Test Accuracy: 0.5503876209259033


In [ ]:
keras_additional_conv = load_model(''.join([project_folder, "/Models/keras_additional_conv.keras"]))
loss, accuracy = keras_additional_conv.evaluate(fakeVReal_TestData, fakeVReal_TestLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9918 - loss: 0.0070
Test Loss: 0.006493733264505863
Test Accuracy: 0.9922480583190918


In [ ]:
keras_imageTransforms = load_model(''.join([project_folder, "/Models/keras_imageTransforms.keras"]))
loss, accuracy = keras_imageTransforms.evaluate(fakeVReal_TestData, fakeVReal_TestLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.9961 - loss: 0.0117
Test Loss: 0.017155881971120834
Test Accuracy: 0.9922480583190918


##### STEP 3: Evaluate the PyTorch Models

In [ ]:
pyTorch_conversion = get_pytorch_ref_model()
pyTorch_conversion.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_conversion.pth"])))
pyTorch_conversion.to(DEVICE)
test_pytorch(pyTorch_conversion)

Test accuracy: 1.0000 - Test loss: 0.0059


In [ ]:
pyTorch_10xLR = get_pytorch_ref_model()
pyTorch_10xLR.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_10X_LR.pth"])))
pyTorch_10xLR.to(DEVICE)
test_pytorch(pyTorch_10xLR)

Test accuracy: 0.9767 - Test loss: 0.0729


In [ ]:
pyTorch_100xLR = get_pytorch_ref_model()
pyTorch_100xLR.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_100X_LR.pth"])))
pyTorch_100xLR.to(DEVICE)
test_pytorch(pyTorch_100xLR)

Test accuracy: 0.9767 - Test loss: 0.0357


In [ ]:
pyTorch_additional_layers = get_pytorch_ref_model_more_layers()
pyTorch_additional_layers.load_state_dict(torch.load(''.join([project_folder, "/Models/pyTorch_additional_conv.pth"])))
pyTorch_additional_layers.to(DEVICE)
test_pytorch(pyTorch_additional_layers)

Test accuracy: 1.0000 - Test loss: 0.0048


In [ ]:
pyTorch_imageTransforms = get_pytorch_ref_model()
pyTorch_imageTransforms.load_state_dict(torch.load(''.join([project_folder, "/Models/pyTorch_imageTransforms.pth"])))
pyTorch_imageTransforms.to(DEVICE)
test_pytorch(pyTorch_imageTransforms)

Test accuracy: 0.9225 - Test loss: 0.1891


## Part 7: Test Models (New Dataset)

##### STEP 1: Setup

1. Reset Parameters to Baseline if needed.

In [ ]:
IMG_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.0001 # Pg. 8 of "Classification of Real and Fake Human Faces Using Deep Learning" by Fatima Maher Salman and Samy S. Abu-Naser
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2. Process the dataset.

In [ ]:
testData, testLabels, testData_T, testLabels_T = basic_process_data(df_TestDataset, TEST_IMAGE_PATH, extension='')

3. Load the Test Dataset for PyTorch

In [ ]:
test_dataset_T = torch.utils.data.TensorDataset(testData_T, testLabels_T)

test_loader = DataLoader(test_dataset_T, batch_size=BATCH_SIZE, shuffle=True, generator=GENERATOR)

##### STEP 2: Evaluate the Keras Models

In [ ]:
keras_reference = load_model(''.join([project_folder, "/Models/model_reference.keras"]))
loss, accuracy = keras_reference.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.3810 - loss: 2.6682
Test Loss: 1.9629323482513428
Test Accuracy: 0.5116279125213623


In [ ]:
keras_10xLR = load_model(''.join([project_folder, "/Models/keras_10x_LR.keras"]))
loss, accuracy = keras_10xLR.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.4066 - loss: 3.0724
Test Loss: 2.670891046524048
Test Accuracy: 0.45348837971687317


In [ ]:
keras_100xLR = load_model(''.join([project_folder, "/Models/keras_100x_LR.keras"]))
loss, accuracy = keras_100xLR.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.7558 - loss: 0.6470
Test Loss: 0.6981244683265686
Test Accuracy: 0.5


In [ ]:
keras_additional_conv = load_model(''.join([project_folder, "/Models/keras_additional_conv.keras"]))
loss, accuracy = keras_additional_conv.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.4134 - loss: 2.6967
Test Loss: 2.1399972438812256
Test Accuracy: 0.4922480583190918


In [ ]:
keras_imageTransforms = load_model(''.join([project_folder, "/Models/keras_imageTransforms.keras"]))
loss, accuracy = keras_imageTransforms.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5756 - loss: 1.5998
Test Loss: 1.7155238389968872
Test Accuracy: 0.5310077667236328


##### STEP 3: Evaluate the PyTorch Models

In [ ]:
pyTorch_conversion = get_pytorch_ref_model()
pyTorch_conversion.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_conversion.pth"])))
pyTorch_conversion.to(DEVICE)
test_pytorch(pyTorch_conversion)

Test accuracy: 0.5310 - Test loss: 3.1531


In [ ]:
pyTorch_10xLR = get_pytorch_ref_model()
pyTorch_10xLR.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_10X_LR.pth"])))
pyTorch_10xLR.to(DEVICE)
test_pytorch(pyTorch_10xLR)

Test accuracy: 0.4845 - Test loss: 3.4627


In [ ]:
pyTorch_100xLR = get_pytorch_ref_model()
pyTorch_100xLR.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_100X_LR.pth"])))
pyTorch_100xLR.to(DEVICE)
test_pytorch(pyTorch_100xLR)

Test accuracy: 0.5000 - Test loss: 2.0829


In [ ]:
pyTorch_additional_layers = get_pytorch_ref_model_more_layers()
pyTorch_additional_layers.load_state_dict(torch.load(''.join([project_folder, "/Models/pyTorch_additional_conv.pth"])))
pyTorch_additional_layers.to(DEVICE)
test_pytorch(pyTorch_additional_layers)

Test accuracy: 0.4806 - Test loss: 1.6503


In [ ]:
pyTorch_imageTransforms = get_pytorch_ref_model()
pyTorch_imageTransforms.load_state_dict(torch.load(''.join([project_folder, "/Models/pyTorch_imageTransforms.pth"])))
pyTorch_imageTransforms.to(DEVICE)
test_pytorch(pyTorch_imageTransforms)

Test accuracy: 0.5271 - Test loss: 1.0268
